In [1]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
import requests
from bs4 import BeautifulSoup

!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")


    Linking successful
    /home/aa5118/anaconda3/envs/plural/lib/python3.7/site-packages/en_core_web_sm
    -->
    /home/aa5118/anaconda3/envs/plural/lib/python3.7/site-packages/spacy/data/en_core_web_sm

    You can now load the model via spacy.load('en_core_web_sm')



In [44]:
class NamedEntityRecognition:
    
    def __init__(self, url):
        
        self.url = url
        self.companies = None
        self.people = None
        self.keywords = None
        self.locations = None
        self.events = None
        self.products = None
    
    def jsonify(self):
        return None

In [47]:
class Extract:
    
    def __init__(self, filename):
        
        sources=pd.read_csv(filename,header=None,names=["url"])
        #sources=sources.append({"url":"www.balsejfnweo.com"}, ignore_index = True) 
        
        self.sources = sources.head()
        self.bad_urls = []
        self.extracted = self.__extract_text()
        self.ner_list = None
        
    def get_number_of_urls(self):
        return len(self.sources)
    
    def __extract_text(self):
        
        extracted=[]
        for url in self.sources['url']:

            url = "http://"+url
            
            try:
                html = requests.get(url)
            except requests.exceptions.RequestException as e:
                self.bad_urls.append(url)
                print (e)
                
            if str(html) == "<Response [200]>":

                soup = BeautifulSoup(html.text)

                # kill all script and style elements
                for script in soup(["script", "style"]):
                    script.extract()    # rip it out

                # get text
                text = soup.get_text()

                # break into lines and remove leading and trailing space on each
                lines = (line.strip() for line in text.splitlines())
                # break multi-headlines into a line each
                chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
                # drop blank lines
                text = '\n'.join(chunk for chunk in chunks if chunk)
            else:
                text=""

            extracted.append(text)

        return extracted
    
    def extract_entities(self):
        
        ner_list=[]
        
        for i, url in enumerate(self.sources['url']):
            ner_list.append(NamedEntityRecognition(url))
            
            doc = nlp(extracted[i])
            for entity in doc.ents:
                if (entity.label_ == "ORG"):
                    ner_list[0].companies = entity.text
            
        
        self.ner_list = ner_list
        return

In [48]:
test=Extract("./sources.txt")

In [49]:
test.bad_urls

[]

In [53]:
test.ner_list[0].url

'www.accrofab.co.uk'